In [24]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings, get_response_synthesizer
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core.query_engine import TransformQueryEngine
from llama_index.core.node_parser.text import SentenceSplitter
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.schema import TextNode
from llama_index.llms.ollama import Ollama
import tqdm

In [6]:
# craete a sentence Splitter to split the documents into sentences with chunk size and overlap to capture a part of the previous sentence
text_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=512)

text_chunks = []                # This will hold all the chunks of text from all documents
doc_idxs = []                   # This will keep track of the document each chunk came from

docs = SimpleDirectoryReader("Backend/uploads/a00f3625-43c8-405b-bf7e-ae4b73b84182").load_data()

for doc_idx, doc in enumerate(docs):
    # Split the current document's text into chunks
    cur_text_chunks = text_parser.split_text(doc.text)

    # Extend the list of all text chunks with the chunks from the current document
    text_chunks.extend(cur_text_chunks)

    # Extend the document index list with the index of the current document, repeated for each chunk
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [7]:
nodes = []                          # This will hold all TextNode objects created from the text chunks

# Iterate over each text chunk and its index
for idx, text_chunk in tqdm.tqdm(enumerate(text_chunks)):
    
    node = TextNode(text=text_chunk)    # Create a TextNode object with the current text chunk
    src_doc = docs[doc_idxs[idx]]       # Retrieve the source document using the current index mapped through doc_idxs
    node.metadata = src_doc.metadata    # Assign the source document's metadata to the node's metadata attribute

    nodes.append(node)                  # Append the newly created node to the list of nodes

7it [00:00, ?it/s]


In [8]:
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
for node in tqdm.tqdm(nodes):
    node_embedding = embed_model.get_text_embedding(node.get_content(metadata_mode="all"))
    node.embedding = node_embedding

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]c:\Users\DELL\anaconda3\envs\llm\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\AppData\Local\Temp\fastembed_cache\models--qdrant--bge-small-en-v1.5-onnx-q. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
100%|██████████| 7/7 [00:01<00:00,  3.

In [20]:
len(nodes[0].embedding)

384

In [18]:
import os
from dotenv import load_dotenv

load_dotenv()

os.getenv['pinecone_api']

TypeError: 'function' object is not subscriptable

In [ ]:
import os
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec



# Create Pinecone Vector Store
pc = Pinecone(api_key='key')

pc.create_index(
    name="a00f3625-43c8-405b-bf7e-ae4b73b84182",
    dimension=384,
    metric="dotproduct",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

pinecone_index = pc.Index("a00f3625-43c8-405b-bf7e-ae4b73b84182")

vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index,
)

In [25]:
llm = Ollama(model="phi3", request_timeout=600)

Settings.transformations = [SentenceSplitter(chunk_size=1024, chunk_overlap=512)]
Settings.embed_model = embed_model
Settings.llm = llm

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context,
    transformations=Settings.transformations,
)

Upserted vectors: 100%|██████████| 7/7 [00:02<00:00,  2.84it/s]


In [26]:
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=5)
response_synthesizer = get_response_synthesizer()
vector_query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=response_synthesizer,
)

In [27]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(vector_query_engine, hyde)

In [28]:
def queries(query_str):
    response = hyde_query_engine.query(query_str)
    print(response)
    return str(response)

def respond(message, chat_history):
        bot_message = queries(message)
        chat_history.append((message, bot_message))
        return "", chat_history

In [31]:
response = hyde_query_engine.query("What is the mailing address of Hassan Ibrar")
print(response)

RemoteProtocolError: Server disconnected without sending a response.